# Msc Thesis

Building an FR model

## LIBRARY IMPORTS

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
from PIL import Image
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

from mtcnn.mtcnn import MTCNN
import cv2
from tqdm.notebook import tqdm

## LOAD DATASET AND PREPROCESS

A few preprocessing steps are required to make the dataset ready for training

- Extrace faces
- Replace images with the extracted faces (This is common and much more efficient than including the whole image, since it can include many unimportant features)
- Rescale the images to a smaller size (also a matter of efficiency. It is handled by the face extraction function formulated below)
- Saving the train and test sets after preprocessing as numpy objects to enable easy reproduction of experiments in the future by just loading the numpy object.
- Creating batch organised datasets


### Face Extraction

In [2]:
# can detect multiple faces but for our tasks we assume just one face images only
def extract_faces_from_img(imagePath, required_size=(160, 160)):
    image = Image.open(imagePath)
    image = np.asarray(image)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    faces = faceCascade.detectMultiScale(
                                        gray,
                                        scaleFactor=1.3,
                                        minNeighbors=3,
                                        minSize=(30, 30)
                                        )
    if len(faces) != 0:
        for (x, y, w, h) in faces:
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2)
            roi_color = image[y:y + h, x:x + w]
        # resize pixels to the model size
        image = Image.fromarray(roi_color)
        image = image.resize(required_size)
        face_array = np.asarray(image)
    else:
        # if face is not detected we need to return None
        face_array = None
    return face_array

## Custom 6-person dataset for a classiffier

In [3]:
samples = {
            'n002462':'Elizabeth Olsen',
            'n002038':'David Hasselhoff',
            'n000419':'Amy Adams',
            'n000709':'Antonio Banderas',
            'n001063':'Billy Idol',
            'n003862':'Jason Derulo'
}

train_ds_path = r'C:\Users\Keravnos\Documents\VGG-Face2\data\vggface2_train.tar\train'

def prepare_dataset(samples, train_data_path):
    ds = []
    # list out dict keys
    key_list = list(samples.keys())
    #iterate over our 6 identity directories
    for k in tqdm(key_list,'Identities'):
        # use this as label
        label = samples[k]
        #directory path of images for each identity
        dir_path = os.path.join(train_data_path,k)
        print('Extracting images for', label)
        # iterate over image directory and extract faces
        for img in tqdm(os.listdir(dir_path),'Images'):
            # need the image path for the face extraction method
            img_path = os.path.join(dir_path,img)
            # create a face array variable where the extracted face is constructed by the function.
            face_array = extract_faces_from_img(img_path)
            # add both image and label in dataset
            ds.append((face_array,label))
    print('DATASET COMPILED!')
    return ds

In [4]:
ds = prepare_dataset(samples,train_ds_path)

Identities:   0%|          | 0/6 [00:00<?, ?it/s]

Extracting images for Elizabeth Olsen


Images:   0%|          | 0/559 [00:00<?, ?it/s]

Extracting images for David Hasselhoff


Images:   0%|          | 0/542 [00:00<?, ?it/s]

...

Extracting images for Billy Idol


Images:   0%|          | 0/396 [00:00<?, ?it/s]

Extracting images for Jason Derulo


Images:   0%|          | 0/508 [00:00<?, ?it/s]

DATASET COMPILED!


In [5]:
ds = np.asarray(ds, dtype=object)
ds.shape

(3145, 2)

Clean all non recognised images from dataset.

In [6]:
ds = [(image,label) for (image,label) in ds if image.__class__.__name__ != 'NoneType' ]

In [7]:
ds = np.asarray(ds, dtype=object)
ds.shape

(2599, 2)

In [8]:
from sklearn.preprocessing import LabelEncoder

images = [image for image,label in ds]
labels = [label for image,label in ds]

# encode class values as integers
encoder = LabelEncoder()
encoder.fit(labels)
encoded_Y = encoder.transform(labels)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = tf.keras.utils.to_categorical(encoded_Y)

SPLIT

In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(images, dummy_y, test_size=0.3, random_state=42)

Preprocess for xception model and prepare batch size

In [24]:
train_dataset = tf.data.Dataset.from_tensor_slices((X_train,y_train))

In [25]:
test_dataset = tf.data.Dataset.from_tensor_slices((X_test,y_test))

In [26]:
def preprocess(image,label):
    image = img_final = tf.cast(image, tf.float32) / 255.0
    final_image = tf.keras.applications.xception.preprocess_input(image)
    return final_image,label

batch_size = 8
train_dataset = train_dataset.shuffle(1000)
train_dataset = train_dataset.map(preprocess).batch(batch_size).prefetch(1)

test_dataset = test_dataset.shuffle(1000)
test_dataset = test_dataset.map(preprocess).batch(batch_size).prefetch(1)


## PREPROCESS DATASET

## BUILDING THE MODEL

Use Transfer Learning to build on top of an Xception model architecture for image classification.

In [18]:
from tensorflow import keras
from tensorflow.keras import layers

n_classes = len(list(samples.keys()))

# data_augmentation = keras.Sequential(
#     [
#         layers.experimental.preprocessing.RandomFlip("horizontal"),
#         layers.experimental.preprocessing.RandomRotation(0.1),
#     ]
# )

base_model = keras.applications.Xception(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(160, 160, 3),
    include_top=False
)  # Do not include the ImageNet classifier at the top.


avg = keras.layers.GlobalAveragePooling2D()(base_model.output)
drop = keras.layers.Dropout(0.2)(avg)  # Regularize with dropout
output = keras.layers.Dense(n_classes, activation='softmax')(drop)
model = keras.Model(base_model.input, output)

# Freeze the base_model
for layer in base_model.layers:
    layer.trainable = False

# Create new model on top

# x = data_augmentation(inputs)  # Apply random data augmentation



model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
block1_conv1 (Conv2D)           (None, 79, 79, 32)   864         input_2[0][0]                    
__________________________________________________________________________________________________
block1_conv1_bn (BatchNormaliza (None, 79, 79, 32)   128         block1_conv1[0][0]               
__________________________________________________________________________________________________
block1_conv1_act (Activation)   (None, 79, 79, 32)   0           block1_conv1_bn[0][0]            
____________________________________________________________________________________________

In [27]:
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint
import math

# optimizer = SGD(learning_rate=0.002, momentum=0.9, decay=0.01)
optimizer = Adam(learning_rate=0.002)

# recompile
model.compile(
    optimizer=optimizer,
    loss='categorical_crossentropy',
    metrics=['accuracy'])

# train
history = model.fit(train_dataset,
                    epochs=20,
                    validation_data=test_dataset,
                    verbose=1,
                    callbacks=[
                                ModelCheckpoint(filepath='model_ckp/model_fine_ep{epoch}.h5'),
                                    ]
)
model.save('model_ckp/model_fine_final.h5')

Epoch 1/20
114/114 [==============================] - 9s 48ms/step - loss: 1.7138 - accuracy: 0.2622 - val_loss: 1.7156 - val_accuracy: 0.2474
Epoch 2/20
114/114 [==============================] - 4s 36ms/step - loss: 1.7188 - accuracy: 0.2705 - val_loss: 1.7157 - val_accuracy: 0.2449
Epoch 3/20
114/114 [==============================] - 4s 36ms/step - loss: 1.7114 - accuracy: 0.2688 - val_loss: 1.7155 - val_accuracy: 0.2462
Epoch 4/20
114/114 [==============================] - 4s 36ms/step - loss: 1.7092 - accuracy: 0.2694 - val_loss: 1.7155 - val_accuracy: 0.2474
Epoch 5/20
114/114 [==============================] - 4s 36ms/step - loss: 1.7177 - accuracy: 0.2589 - val_loss: 1.7154 - val_accuracy: 0.2474
Epoch 6/20
114/114 [==============================] - 4s 36ms/step - loss: 1.7145 - accuracy: 0.2617 - val_loss: 1.7156 - val_accuracy: 0.2462
Epoch 7/20
114/114 [==============================] - 4s 36ms/step - loss: 1.7130 - accuracy: 0.2683 - val_loss: 1.7155 - val_accuracy: 0.2462

In [183]:
# unfreeze and retrain
for layer in base_model.layers:
    layer.trainable = True

In [ ]:
# for x,y in test:
#     print(y)

In [26]:
test_path = r'C:\Users\Keravnos\Documents\VGG-Face2\data\vggface2_train.tar\train\n002038\0107_01.jpg'
load_test_face = extract_faces_from_img(test_path)
lbl = ''

print(load_test_face.shape)
img, lbl = preprocess(load_test_face,lbl)
test_img_ds = tf.data.Dataset.from_tensor_slices(img)
pred = model.predict(test_img_ds, batch_size=None)

(160, 160, 3)


ValueError: in user code:

    C:\Users\Keravnos\Documents\AdversarialFR\venv\lib\site-packages\tensorflow\python\keras\engine\training.py:1569 predict_function  *
        return step_function(self, iterator)
    C:\Users\Keravnos\Documents\AdversarialFR\venv\lib\site-packages\tensorflow\python\keras\engine\training.py:1559 step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    C:\Users\Keravnos\Documents\AdversarialFR\venv\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:1285 run
        return self._extended.call_for_each_replica(fn, args=args, kwargs=kwargs)
    C:\Users\Keravnos\Documents\AdversarialFR\venv\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:2833 call_for_each_replica
        return self._call_for_each_replica(fn, args, kwargs)
    C:\Users\Keravnos\Documents\AdversarialFR\venv\lib\site-packages\tensorflow\python\distribute\distribute_lib.py:3608 _call_for_each_replica
        return fn(*args, **kw

In [209]:
print(pred)

NameError: name 'pred' is not defined